In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/home/yuanz/documents/weights/Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype="auto", device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)



# Keep the process alive for a while to see the output


In [2]:
prompt = "介绍一下你自己"


def build_model_inputs(prompt):
    messages = [
        {
            "role": "system",
            "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.",
        },
        {"role": "user", "content": prompt},
    ]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    return model_inputs


model_inputs = build_model_inputs(prompt)
model_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [3]:
model_outputs = model(**model_inputs)

In [6]:
def show_model_outputs(model_outputs):
    logits = model_outputs.logits
    logits_info = {
        "shape": logits.shape,
        'dtype': logits.dtype,
        'logits_mean':logits.mean(axis=1)[0, :3].tolist(),  # Convert to list for better readability
        "value": logits[0, 0, :10].tolist(),  # Convert to list for better readability
        "grad_enabled": logits.requires_grad,
    }
    return logits_info


show_model_outputs(model_outputs)


{'shape': torch.Size([1, 31, 151936]),
 'dtype': torch.bfloat16,
 'logits_mean': [4.875, 4.375, 2.984375],
 'value': [3.96875,
  7.8125,
  3.296875,
  -0.89453125,
  6.65625,
  3.734375,
  4.5,
  6.6875,
  8.5625,
  6.375],
 'grad_enabled': True}

In [5]:
model.lm_head.weight.data[0, :3].tolist()

[-0.00506591796875, -0.006439208984375, -0.00787353515625]

In [ ]:
model.lm_head.weight.shape, model.lm_head.weight

In [ ]:
model.get_input_embeddings().weight.shape, model.get_input_embeddings().weight

In [ ]:
model.config.tie_word_embeddings
